In [4]:
!pip list

Package                       Version
----------------------------- --------
absl-py                       1.4.0
accelerate                    0.22.0
aiohttp                       3.8.3
aiosignal                     1.3.1
asttokens                     2.2.1
astunparse                    1.6.3
async-timeout                 4.0.2
attrs                         22.2.0
backcall                      0.2.0
backports.functools-lru-cache 1.6.5
cachetools                    5.3.1
certifi                       2023.5.7
charset-normalizer            2.1.1
click                         8.1.5
colorama                      0.4.6
comm                          0.1.4
contourpy                     1.1.0
cycler                        0.11.0
Cython                        0.29.33
datasets                      2.14.4
debugpy                       1.5.1
decorator                     5.1.1
dill                          0.3.7
evaluate                      0.4.0
exceptiongroup                1.1.3
executing     

In [5]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import torch
torch.cuda.is_available()

False

In [2]:
!pip install evaluate

In [4]:
# Помогает решить ошибки во время импорта "transformers"
!pip install nltk
!pip install --upgrade accelerate

In [6]:
!pip install setuptools=="67.8.0"

     ---------------------------------------- 1.1/1.1 MB 1.2 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.0.0
    Uninstalling setuptools-68.0.0:
      Successfully uninstalled setuptools-68.0.0


In [7]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import os
import re
import torch
import evaluate
from sklearn import preprocessing
import transformers
from transformers import AutoTokenizer
from transformers import Trainer
from transformers import TrainingArguments
from transformers import (
    AutoConfig,
    AutoModel,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    DataCollatorWithPadding,
    PretrainedConfig,
    default_data_collator   
)

import datasets
from datasets import load_dataset
from datasets import load_dataset_builder
from datasets import Dataset

torch.cuda.is_available()
os.environ["WANDB_DISABLED"] = "true"

stemmer = PorterStemmer()

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

AttributeError: module 'distutils' has no attribute 'version'

In [5]:
df_train = pd.read_csv('train.csv').dropna()
df_test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')
print('Shape train: {}'.format(df_train.shape))
print('Shape test: {}'.format(df_test.shape))
print('Shape sample_submission: {}'.format(sample_submission.shape))
print('-'*40)
print('Gaps train: {}'.format(df_train.isnull().sum().sum()))
print('Gaps test: {}'.format(df_test.isnull().sum().sum()))
print('Gaps sample_submission: {}'.format(sample_submission.isnull().sum().sum()))

Shape train: (41155, 3)
Shape test: (3798, 2)
Shape sample_submission: (3798, 2)
----------------------------------------
Gaps train: 0
Gaps test: 0
Gaps sample_submission: 0


In [6]:
df_train.sample(5)

,Unnamed: 0,Text,Sentiment
15387,15387,Scammers are taking advantage of fears surroun...,Positive
35761,35760,Our latest COVID 19 Fraud Watch Group update h...,Negative
28144,28144,Imagine corporations like recognized the value...,Positive
16696,16696,M sure this is now punishable by death!#corona...,Extremely Negative
31919,31919,East African holding is shouldering real citiz...,Extremely Negative


In [7]:
LABELS = df_train['Sentiment'].unique()
LABELS_COUNT = len(df_train['Sentiment'].unique())

# EDA

In [8]:
# Помогает решить ошибки для "wordnet"
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [8]:
def preprocess_text(text, url=False, lower=True, punct=True, stop=True, stem=False, lem=True):

    # Очистка текста от URL-адресов
    if url:
        text = re.sub(r'http\S+', '', text)
    
    # Приведение символов к нижнему регистру
    if lower:
        text = text.lower()

    # Удаление пунктуации
    if punct:
        text = text.translate(str.maketrans("", "", string.punctuation))

    # Удаление стоп-слов
    if stop:
        stop_words = set(stopwords.words('english'))
        words = nltk.word_tokenize(text)
        filtered_words = [word for word in words if word not in stop_words]
        text = " ".join(filtered_words)

    # Стемминг
    if stem:
        porter_stemmer = PorterStemmer()
        words = nltk.word_tokenize(text)
        stemmed_words = [porter_stemmer.stem(word) for word in words]
        text = " ".join(stemmed_words)

    # Лемматизация
    if lem:
        wordnet_lemmatizer = WordNetLemmatizer()
        words = nltk.word_tokenize(text)
        lemmatized_words = [wordnet_lemmatizer.lemmatize(word) for word in words]
        text = " ".join(lemmatized_words)

    return text

In [9]:
df_train['Text'] = df_train['Text'].apply(preprocess_text)
df_test['Text'] = df_test['Text'].apply(preprocess_text)

In [10]:
df_train.sample(3)

,Unnamed: 0,Text,Sentiment
38325,38324,dear fellow citizen reasi team reasi stayhomes...,Positive
4908,4908,covid19 shii getting real protect family much ...,Positive
13896,13896,whatever theyre paying people work supermarket...,Extremely Negative


In [11]:
# Закодируем признак `Sentiment`
le = preprocessing.LabelEncoder()
le.fit(LABELS)

df_train['Sentiment'] = le.transform(df_train['Sentiment'])
df_train.rename(columns={'Sentiment': 'label'}, inplace=True)

In [12]:
# tokenizer
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# model
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,
                                                           num_labels=LABELS_COUNT,
                                                           ignore_mismatched_sizes=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [13]:
# Запишем функцию
def tokenize_function(data):
    return tokenizer(data["Text"], truncation=True)

In [14]:
df_test.head()

,id,Text
0,787bc85b-20d4-46d8-84a0-562a2527f684,trending new yorkers encounter empty supermark...
1,17e934cd-ba94-4d4f-9ac0-ead202abe241,couldnt find hand sanitizer fred meyer turned ...
2,5914534b-2b0f-4de8-bb8a-e25587697e0d,find protect loved one coronavirus
3,cdf06cfe-29ae-48ee-ac6d-be448103ba45,panic buying hit newyork city anxious shopper ...
4,aff63979-0256-4fb9-a2d9-86a3d3ca5470,toiletpaper dunnypaper coronavirus coronavirus...


In [15]:
# Train
train_dt = Dataset.from_pandas(df_train[['Text','label']])
tokenized_train = train_dt.map(tokenize_function, batched=True)
tokenized_train = tokenized_train.remove_columns(["__index_level_0__"])

#test 
test_dt = Dataset.from_pandas(df_test)
tokenized_test = test_dt.map(tokenize_function, batched=True)
tokenized_test = tokenized_test.remove_columns(['id', 'Text'])


data_collator = DataCollatorWithPadding(tokenizer=tokenizer) #padding funct.

Map:   0%|          | 0/41155 [00:00<?, ? examples/s]

Map:   0%|          | 0/3798 [00:00<?, ? examples/s]

In [16]:
splitted_data= tokenized_train.train_test_split(test_size=0.2)
splitted_data

DatasetDict({
    train: Dataset({
        features: ['Text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 32924
    })
    test: Dataset({
        features: ['Text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8231
    })
})

# Evaluate

In [17]:
# metrics count function
def compute_metrics(eval_preds):
    metric = evaluate.load('accuracy')
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [18]:
training_args = TrainingArguments(
    "test-trainer",     
    evaluation_strategy="epoch",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=32,
    optim="adamw_torch",
    warmup_steps=600,
    weight_decay=0.01,
    logging_steps=1)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [19]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=splitted_data['train'],
    eval_dataset=splitted_data['test'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [20]:
transformers.logging.set_verbosity('CRITICAL')
trainer.train()

  0%|          | 0/6174 [00:00<?, ?it/s]

{'loss': 1.5806, 'learning_rate': 8.333333333333334e-08, 'epoch': 0.0}
{'loss': 1.6981, 'learning_rate': 1.6666666666666668e-07, 'epoch': 0.0}


KeyboardInterrupt: 

## Predict

In [24]:
def get_prediction(text):
    inputs = tokenizer(text, truncation=True,padding=True, return_tensors="pt")#.to("cuda")
    outputs = model(**inputs)
    proba = outputs[0].softmax(1)
    return proba.argmax().item()

In [22]:
trainer.evaluate(tokenized_test)

  0%|          | 0/119 [00:00<?, ?it/s]

{'eval_runtime': 207.7915, 'eval_samples_per_second': 18.278, 'eval_steps_per_second': 0.573, 'epoch': 0.0}


{'eval_runtime': 207.7915,
 'eval_samples_per_second': 18.278,
 'eval_steps_per_second': 0.573,
 'epoch': 0.0}

In [25]:
predictions = df_test['Text'].apply(lambda text: get_prediction(text))
sample_submission['Sentiment'] = le.inverse_transform(predictions)
sample_submission.sample(6)

KeyboardInterrupt: 